In [51]:
import os
import re
import json
import dotenv
from pathlib import Path
from battetl import BattETL

## Set env

In [52]:
dotenv.load_dotenv()
print(f'Check target database "{os.getenv("DB_TARGET")}"')

Check target database "battdb"


## Create config

In [50]:
DATA_FOLDER = os.path.abspath('')
print(f'Check data folder "{DATA_FOLDER}"')

data_files = []
stats_files = []
schedule_files = []
# list files in data folder
for file in os.listdir(DATA_FOLDER):
    # Maccor: Check if file name ends with number.txt
    if re.search(r'\d+\.txt$', file):
        data_files.append(os.path.join(DATA_FOLDER, file))

    # Maccor: Check if file name ends with [STATS].txt
    elif re.search(r'\[STATS\]\.txt$', file):
        stats_files.append(os.path.join(DATA_FOLDER, file))

    # Maccor: Check if file name ends with .000
    elif re.search(r'\.000$', file):
        schedule_files.append(os.path.join(DATA_FOLDER, file))

    # Check: Check if Arbin data file
    elif ('Wb' in re.split('_',file) and re.split('_',file)[-1].endswith('.CSV')):
        data_files.append(os.path.join(DATA_FOLDER, file))

    # Arbin : Stats file
    elif re.search(r'StatisticByCycle.CSV$', file):
        stats_files.append(os.path.join(DATA_FOLDER, file))

    # Maccor: Check if file schedule file
    elif re.search(r'.sdx$', file):
        schedule_files.append(os.path.join(DATA_FOLDER, file))

# Sort files by name, ascending
data_files.sort(reverse=False)
stats_files.sort(reverse=False)

print(f'>> Found {len(data_files)} data files.')
print('\n'.join(data_files))
print(f'>> Found {len(stats_files)} stats files.')
print('\n'.join(stats_files))
print(f'>> Found {len(schedule_files)} schedule files.')
print('\n'.join(schedule_files))

Check data folder "/Users/zander/Desktop/battetl/tests/data"
>> Found 0 data files.

>> Found 0 stats files.

>> Found 0 schedule files.



In [7]:
config = {
    "timezone": "America/Los_Angeles",
    "data_file_path": data_files,
    "stats_file_path": stats_files,
    "schedule_file_path": schedule_files,
    "target_databases": [
        os.getenv("DB_TARGET"),
    ],
    "meta_data": {
        "test_meta": {
            "cell_id": None,
            "schedule_id": None,
            # Get test name from first data file. The name is before first space.
            "test_name": os.path.basename(data_files[0]).split(' ', 1)[0],
            "start_date": '2020-10-06',
            "end_date": '2020-10-11',
            # Get the channel number from the first data file. The number is before the first dot.
            "channel": int(os.path.basename(data_files[0]).split('.', 1)[0].split(' ')[-1]),
            "ev_chamber": 6,
            "ev_chamber_slot": 12,
            "thermocouples": None,
            "thermocouple_channels": None,
            "comments": "Ran at 45 degrees C",
            "project_id": None,
            "test_capacity_mah": 2650,
            "potentiostat_id": None,
            "cycler_id": None,
        },
        "cell": {
            "cell_type_id": None,
            "batch_number": "BATCH_NUMBER", 
            "label": "24",
            "date_received": "2020-09-01",
            "comments": None,
            "date_manufactured": None,
            "manufacturer_sn": "BattGenie_SN",
            "dims": None,
            "weight_g": None,
            "first_received_at_voltage_mv": None,
        },
        "cell_meta": {
            "manufacturer": "BattGenie",
            "manufacturer_pn": "BattGenie_PN",
            "form_factor": "pouch",
            "capacity_mah": 2720,
            "chemistry": None,
            "dimensions": '{"x_mm":"54.25", "y_mm":106.96, "z_mm":3.19}',
        },
        "schedule_meta": {
            "schedule_name": "BG_Characterization_v1",
            "test_type": "Characterization",
            "cycler_make": "Maccor",
            "date_created": "2020-10-06",
            "created_by": "BattGenie",
            "comments": None,
            "cv_voltage_threshold_mv": None,
            "schedule_files": None,
        },
        "cycler": {
            "sn": "SN",
            "calibration_date": None,
            "calibration_due_date": None,
            "location": "BattGenie",
            "timezone_based": None,
        },
        "cycler_meta": {
            "manufacturer": "Maccor",
            "model": "SERIES 4000M",
            "datasheet": None,
            "num_channels": None,
            "lower_current_limit_a": None,
            "upper_current_limit_a": None,
            "lower_voltage_limit_v": None,
            "upper_voltage_limit_v": None,
        }
    }
}

# Deep remove None values from config
config = {k: v for k, v in config.items() if v is not None}
config['meta_data'] = {k: v for k, v in config['meta_data'].items() if v is not None}
config['meta_data']['test_meta'] = {k: v for k, v in config['meta_data']['test_meta'].items() if v is not None}
config['meta_data']['cell'] = {k: v for k, v in config['meta_data']['cell'].items() if v is not None}
config['meta_data']['cell_meta'] = {k: v for k, v in config['meta_data']['cell_meta'].items() if v is not None}
config['meta_data']['schedule_meta'] = {k: v for k, v in config['meta_data']['schedule_meta'].items() if v is not None}
config['meta_data']['cycler'] = {k: v for k, v in config['meta_data']['cycler'].items() if v is not None}
config['meta_data']['cycler_meta'] = {k: v for k, v in config['meta_data']['cycler_meta'].items() if v is not None}

# Save config to file
with open('config.json', 'w') as f:
    json.dump(config, f, indent=4)

## BattETL

In [ ]:
# Load config from file
cell = BattETL(
    config_path='config.json',
    env_path=Path.cwd().parent.parent / '.env',
)
cell.extract().transform().load()